In [34]:
import sys
sys.path.append('../')  # Add the root directory to the system path

import pandas as pd
import json
from diversity_measures.utils import DiversityCalculator, PromptSelection
from diversity_measures.parser import extract_last_letters

# ... rest of the code ...


In [35]:
def read_jsonl(file_name):
    data = []
    with open(file_name, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [36]:
diversity_calculator = DiversityCalculator()
prompt_selector = PromptSelection()
all_scores = []
results = []

In [37]:
# Loop through the 10 jsonl files
for i in range(10):
    file_name = f"./responses/sample_{i}.jsonl"
    data = read_jsonl(file_name)
    
    for entry in data:
        responses = [choice["message"]["content"] for choice in entry["choices"]]
        cleaned_responses = [extract_last_letters(resp) for resp in responses]
        diversity_scores = diversity_calculator.calculate(cleaned_responses, measures=['entropy', 'gini'])
        
        # Append the diversity scores along with the question ID and sample number
        results.append({
            'question_id': entry['question_id'],
            'sample': i,
            'entropy': diversity_scores['entropy'],
            'gini': diversity_scores['gini']
        })

FileNotFoundError: [Errno 2] No such file or directory: './example/responses/sample_0.jsonl'

In [ ]:
# Convert results to DataFrame
df = pd.DataFrame(results)

# Group by question_id and find max/min entropy
max_entropy = df.groupby('question_id')['entropy'].max().reset_index()
min_entropy = df.groupby('question_id')['entropy'].min().reset_index()

# Rename columns for clarity
max_entropy.columns = ['question_id', 'Max Entropy']
min_entropy.columns = ['question_id', 'Min Entropy']

# Merge the max and min entropy DataFrames on question_id
final_df = pd.merge(max_entropy, min_entropy, on='question_id')

final_df

,Measure,Value,Sample
0,Max Entropy,4.321928,0
1,Max Entropy,4.321928,1
2,Max Entropy,4.321928,2
3,Max Entropy,4.221928,3
4,Max Entropy,4.321928,4
5,Max Entropy,4.121928,5
6,Max Entropy,4.121928,6
7,Max Entropy,4.321928,7
8,Max Entropy,4.321928,8
9,Max Entropy,4.321928,9
